In [ ]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as bs

In [ ]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree

### 1. 판례 목록 나와있는 url 가져오기
> 입력 변수
* OC	string(필수)	사용자 이메일의 ID(g4c@korea.kr일경우 OC값=g4c)
* target	string : prec(필수)	서비스 대상
* type	char(필수)	출력 형태 : HTML / XML
* search	int	검색범위 (기본 : 1 판례명) 2 : 본문검색
* query	string	검색범위에서 검색을 원하는 질의(검색 결과 리스트)
* display	int	검색된 결과 개수 (default=20 max=100)
* page	int	검색 결과 페이지 (default=1)
* org	string	법원종류 (대법원:400201, 하위법원:400202)
* curt	string	법원명 (대법원, 서울고등법원, 광주지법, 인천지방법원)
* JO	string	참조법령명(형법, 민법 등)
* gana	string	사전식 검색(ga,na,da…,etc)
* sort	string	정렬옵션
* lasc : 사건명 오름차순
* ldes : 사건명 내림차순
* dasc : 선고일자 오름차순
* ddes : 선고일자 내림차순(생략시 기본)
* nasc : 법원명 오름차순
* ndes : 법원명 내림차순
* date	int	판례 선고일자
* prncYd	string	선고일자 검색(20090101~20090130)
* nb	int	판례 사건번호
* popYn	string	상세화면 팝업창 여부(팝업창으로 띄우고 싶을 때만 'popYn=Y')
> ##### (필수) 제외하면 안들어가도 되는 변수임
</br>

* #### 목록 parameter 딕셔너리


> query가 keyword에 해당하는데, input 함수로 키워드를 입력받음
<br/> 여기선 키워드로 손해배상 입력, 보여줄 판례의 수는 10개, 법원은 대법원, 기간은 20100101~20191231로 설정함
<br/> type은 url 타고 웹에서 볼거면 html로 하는게 낫고, 주피터노트북으로 text 확인할거면 xml이 나음

In [ ]:
chart= {'OC':'ICTPoolC'
        ,'target':'prec'
        ,'type':'XML'
        ,'query':input()
        ,'display':10000
        ,'curt':'대법원'
        ,'prncYd':'20100101~20191231'
       }

손해배상


* #### request 모듈을 이용하여 chart의 변수들을 전송함

In [ ]:
res_chart=rq.get('http://www.law.go.kr/DRF/lawSearch.do?'
            ,params=chart)

* #### url 가져오기

In [ ]:
print(res_chart.url)

http://www.law.go.kr/DRF/lawSearch.do?OC=ICTPoolC&target=prec&type=XML&query=%EC%86%90%ED%95%B4%EB%B0%B0%EC%83%81&display=10000&curt=%EB%8C%80%EB%B2%95%EC%9B%90&prncYd=20100101~20191231


* #### text 가져오기
> 아래에서 볼 수 있듯이 html 타입으로 text 읽어오는건 무의미한듯..

In [ ]:
print(res_chart.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko">
<head>
<meta http-equiv="X-UA-Compatible" content="IE=edge">      
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">  
<link rel="stylesheet" type="text/css" href="/DRF/css/common.css">
<script type="text/javascript" src="/DRF/js/common/jquery/jquery.js"></script>
	<script type="text/javascript" src="/DRF/js/common/jquery/jquery.form.js"></script>
	<script type="text/javascript" src="/DRF/js/ext/adapter/ext/ext-base.js"></script>
	<script type="text/javascript" src="/DRF/js/ext/ext-all.js"></script>
	<script type="text/javascript" src="/DRF/js/common.js"></script>
	<script type="text/javascript" src="/DRF/js/common-ex.js"></script>
	<script type="text/javascript" src="/DRF/js/CookieUtil.js"></script>
<title>국가법령정보 공동활용</title>
</head> 
<body> 
<!-- //error500 -->
<div id="error500"

### 2. 판례 본문 url 가져오기
> 입력 변수
* OC	string(필수)	사용자 이메일의 ID(g4c@korea.kr일경우 OC값=g4c)
* target	string : prec(필수)	서비스 대상
* type	char(필수)	출력 형태 : HTML / XML
* ID	char(필수)	판례 일련번호
* LM	string	판례명
> ##### (필수) 제외하면 안들어가도 되는 변수임
</br>

* #### 본문 parameter 딕셔너리


> ID가 판례일련번호에 해당하는데, input 함수로 판례일련번호를 입력받음

In [ ]:
main = {'OC':'ICTPoolC',
           'target':'prec',
           'ID': int(input()),
          'type':'XML'}

KeyboardInterrupt: ignored

* #### request 모듈을 이용하여 main의 변수들을 전송함

In [ ]:
res_main= rq.get('http://www.law.go.kr/DRF/lawService.do?',params=main)

* #### url 가져오기

In [ ]:
print(res_main.url)

### 3. 판례 목록에서 판례일련번호만 리스트에 저장하기
> tag로부터 판례일련번호를 가져와야하므로 beautifulSoup 모듈이용

In [ ]:
soup = bs(res_chart.text,'xml')

In [ ]:
find = soup.find_all('판례일련번호')

In [ ]:
find

[]

#### 번호만 가져와야 하므로 slicing 이용

In [ ]:
num_list = []
for k in find:
    line = str(k)
    num_list.append(int(line[8:14]))

In [ ]:
num_list

### 4. 함수
> 세가지의 함수를 만들었다
> * 키워드와 기간을 입력하면, 해당 판례들의 일련번호를 리스트로 반환하는 함수
> * 위의 함수를 이용해 일련번호 리스트 안의 각 번호에 해당하는 판례의 url을 제공하는 함수
> * New! 키워드와 기간을 입력하면 해당 기간의 해당 키워드를 가진 판례의 수를 반환하는 함수

In [ ]:
def get_numbers(keyword, start, end):
    chart= {'OC':'ICTPoolC'
        ,'target':'prec'
        ,'type':'XML'
        ,'query':keyword
        ,'display':10000
        ,'curt':'대법원'
        ,'prncYd':'{0}~{1}'.format(start, end)
       }
    res_chart=rq.get('http://www.law.go.kr/DRF/lawSearch.do?',params=chart)
    soup = bs(res_chart.text,'xml')
    find = soup.find_all('판례일련번호')
    num_list = []
    for k in find:
        line = str(k)
        num_list.append(int(line[8:14]))
    return num_list

In [ ]:
a = get_numbers('손해배상',20100101,20201231)

In [ ]:
len(a)

In [ ]:
def count_case(keyword,start, end):
    num_list = get_numbers(keyword,start,end)
    return len(num_list)

In [ ]:
count_case('손해배상',20100101,20101231)

In [ ]:
def get_url(keyword, start, end):
    num_list = get_numbers(keyword,start, end)
    url_list = []
    for i in num_list:
        main = {'OC':'ICTPoolC',
           'target':'prec',
           'ID': i,
          'type':'HTML'}
        res_main= rq.get('http://www.law.go.kr/DRF/lawService.do?',params=main)
        url_list.append(res_main.url)
    for url in url_list:
        print(url)

In [ ]:
get_url('손해배상', 20100101, 20100331)